# Scrape wikipedia page.

Use the Notebook to build the code to scrape the following Wikipedia page,
- https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Imports

In [73]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import matplotlib.cm as cm
import matplotlib.colors as colors

### Scrape table using pandas

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).content
codes=pd.read_html(results)[0]

### Clean Not assigned boroughs.

In [86]:
p_codes=codes[codes['Borough']!='Not assigned']
p_codes.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Remove duplicate Postcode

In [94]:
strJoin = lambda x:",".join(x.astype(str))  
asIs = lambda x: x  
temp1=p_codes.groupby('Postcode')['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()

temp2=p_codes.groupby('Postcode').first().reset_index()[['Postcode','Borough']]
#temp2=temp

temp3=temp2.join(temp1.set_index('Postcode'), on='Postcode')

p_codes=temp3
p_codes.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Check if Not assigned neighbourhood exist

In [95]:
test=p_codes[p_codes['Neighbourhood']=='Not assigned']
print(test.count())

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64


In [96]:
p_codes.shape

(103, 3)

In [97]:
#!pip install geocoder
#import geocoder # import geocoder

In [98]:
# loop until you get the coordinates
#column_names=['Postcode','Lattitude','Longitude']
#latlong=pd.DataFrame(columns=column_names);
#for postal_code in p_codes['Postcode']:
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#      lat_lng_coords = g.latlng
#    print(lat_lng_coords)
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    latlong.append({'Postcode':postal_code,'Lattitude':latitude,'Longitude':longitude},ignore_index=True)

# Get lattitude and longitude coodinates.

In [99]:
csv_path='http://cocl.us/Geospatial_data'
lat_long=pd.read_csv(csv_path)
lat_long.rename(columns={'Postal Code':'Postcode'},inplace=True)
lat_long.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge lattitude and longitude to neighbourhood dataframe.

In [100]:
merged=p_codes.join(lat_long.set_index('Postcode'),on='Postcode')
merged=merged.reset_index(drop=True)
merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [101]:
merged=merged[merged['Borough'].str.contains('Toronto')]

### Use geopy lib to find lat/long.

In [102]:
from geopy.geocoders import Nominatim 
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Render map of toronto

In [103]:
# create map of New York using latitude and longitude values
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged['Latitude'], merged['Longitude'], merged['Borough'], merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore tornonto neighbourhood

In [104]:
CLIENT_ID = 'XVUNGOG2KNXU40VWE105YA04HD4QFUFMIYVD5DHGTAAWNPPB' # your Foursquare ID
CLIENT_SECRET = 'E4TGRIUOEXTSKPEGRTP41BX0AT5OYVSO0C01HMN5E3WBKQJR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius=500
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XVUNGOG2KNXU40VWE105YA04HD4QFUFMIYVD5DHGTAAWNPPB
CLIENT_SECRET:E4TGRIUOEXTSKPEGRTP41BX0AT5OYVSO0C01HMN5E3WBKQJR


In [105]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [106]:
# type your answer here

toronto_venues = getNearbyVenues(names=merged['Neighbourhood'],
                                   latitudes=merged['Latitude'],
                                   longitudes=merged['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [56]:
print(toronto_venues.shape)
toronto_venues.head()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(3249, 7)
There are 238 uniques categories.


In [85]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,18,18,18,18,18,18
Berczy Park,57,57,57,57,57,57
Brockton,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,35,35,35,35,35,35


### Analyze neighbourhood

In [61]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Take mean of categories

In [62]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,...,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.0,0.01,0.000000
1,Bathurst Quay,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.0,...,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000
2,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.00,0.017544,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000
3,Brockton,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Underground city,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.0,...,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.0,0.00,0.000000
69,Union Station,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.0,0.00,0.000000
70,University of Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.00,0.000000,0.028571,0.0,0.00,0.0,0.0,0.00,0.028571
71,Victoria Hotel,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.0,...,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.0,0.00,0.000000


#### Print top 5 venues in neighbourhoot

In [64]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2   Thai Restaurant  0.04
3              Café  0.04
4  Sushi Restaurant  0.03


----Bathurst Quay----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3       Coffee Shop  0.06
4   Harbor / Marina  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1            Beer Bar  0.04
2      Farmers Market  0.04
3        Cocktail Bar  0.04
4  Seafood Restaurant  0.04


----Brockton----
                venue  freq
0                Café  0.14
1         Coffee Shop  0.09
2      Breakfast Spot  0.09
3  Italian Restaurant  0.05
4        Climbing Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Gym / Fitness Center  0.06
1               Brewery  0.06
2                   Spa  0.06
3            Skate Park  0.06
4            Restaurant  0.06


----CN Tower----
           

               venue  freq
0       Home Service   0.5
1             Garden   0.5
2  Afghan Restaurant   0.0
3        Music Venue   0.0
4     Massage Studio   0.0


----Runnymede----
                venue  freq
0                Café  0.08
1         Pizza Place  0.08
2    Sushi Restaurant  0.06
3  Italian Restaurant  0.06
4         Coffee Shop  0.06


----Ryerson----
                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.08
2        Japanese Restaurant  0.03
3  Middle Eastern Restaurant  0.03
4                       Café  0.03


----South Hill----
                venue  freq
0                 Pub  0.13
1         Coffee Shop  0.13
2  Light Rail Station  0.07
3         Pizza Place  0.07
4        Liquor Store  0.07


----South Niagara----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3       Coffee Shop  0.06
4   Harbor / Marina  0.06


----St. James Town----
            venue  freq
0   

### create pandas data frame with top 10 venues

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Thai Restaurant,Café,Bar,Sushi Restaurant,Gym,Concert Hall,Bakery,Burger Joint
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
2,Berczy Park,Coffee Shop,Restaurant,Bakery,Cheese Shop,Seafood Restaurant,Farmers Market,Café,Cocktail Bar,Beer Bar,Hotel
3,Brockton,Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Skate Park,Light Rail Station,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant


### Create clusters of k=5 and includes top 10 venues

In [70]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

In [71]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Theater,Breakfast Spot,Restaurant,Shoe Store
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Park,Burger Joint,Yoga Studio,Discount Store,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Japanese Restaurant
12,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Diner,Ramen Restaurant,Electronics Store
13,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Diner,Ramen Restaurant,Electronics Store
26,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Diner,Park,Hotel,Breakfast Spot,Japanese Restaurant


In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Observe clusters

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
163,Downtown Toronto,CN Tower,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
164,Downtown Toronto,Bathurst Quay,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
165,Downtown Toronto,Island airport,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
166,Downtown Toronto,Harbourfront West,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
167,Downtown Toronto,King and Spadina,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
168,Downtown Toronto,Railway Lands,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane
169,Downtown Toronto,South Niagara,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Coffee Shop,Plane


Cluster 1 - Area around ariport and related services.

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]




,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Harbourfront,1,Coffee Shop,Bakery,Café,Pub,Park,Mexican Restaurant,Theater,Breakfast Spot,Restaurant,Shoe Store
5,Downtown Toronto,Queen's Park,1,Coffee Shop,Park,Burger Joint,Yoga Studio,Discount Store,Portuguese Restaurant,Nightclub,Mexican Restaurant,Juice Bar,Japanese Restaurant
12,Downtown Toronto,Ryerson,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Diner,Ramen Restaurant,Electronics Store
13,Downtown Toronto,Garden District,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Diner,Ramen Restaurant,Electronics Store
26,Downtown Toronto,St. James Town,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Diner,Park,Hotel,Breakfast Spot,Japanese Restaurant
35,East Toronto,The Beaches,1,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
36,Downtown Toronto,Berczy Park,1,Coffee Shop,Restaurant,Bakery,Cheese Shop,Seafood Restaurant,Farmers Market,Café,Cocktail Bar,Beer Bar,Hotel
40,Downtown Toronto,Central Bay Street,1,Coffee Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint,Café,Ice Cream Shop,Sushi Restaurant,Department Store
41,Downtown Toronto,Christie,1,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Restaurant,Baby Store,Diner,Athletics & Sports
48,Downtown Toronto,Adelaide,1,Coffee Shop,Restaurant,Thai Restaurant,Café,Bar,Sushi Restaurant,Gym,Concert Hall,Bakery,Burger Joint


Cluster 2 - This area popular Cafe, bar and restorant.

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,Central Toronto,Forest Hill North,2,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
122,Central Toronto,Forest Hill West,2,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


 Cluster 3 : Jewellary shops are popular here.

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
111,Central Toronto,Lawrence Park,3,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
149,Central Toronto,Moore Park,3,Gym,Restaurant,Tennis Court,Park,Colombian Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
150,Central Toronto,Summerhill East,3,Gym,Restaurant,Tennis Court,Park,Colombian Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
182,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


Cluster 4 - Sports and activity.

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
112,Central Toronto,Roselawn,4,Home Service,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 5 - Home servies